In [471]:
from collections import Counter

def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    q0=[1, 0] #prepare the zero state
    if num_qubits>1: 
        q1=[1, 0] #prepare zero state again
        for i in range(num_qubits-1): #repeat process with the number of qubits
            q1=np.tensordot(q1,q0,0) #tensor product two the zero states
        q0=q1.flatten() # flatten the qubit
    
    return q0

def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits

    #Pauli gate
    x=np.array([[0, 1],[1, 0]])
    y=np.array([[0, -complex(0,1)],[complex(0,1), 0]])
    z=np.array([[1, 0],[0, -1]])
    #hadmard gate
    h=1/(2**0.5)*np.array([[1, 1],[1, -1]])
    #phase
    s=np.array([[1, 0],[0, complex(0,1)]])
    #identity 
    I = np.identity(2) 

    #Controllnot
    # Define projection operator |0><0|
    P0x0 = np.array([[1, 0],[0, 0]])
    # Define projection operator |1><1|
    P1x1 = np.array([[0, 0],[0, 1]])


    #Single Gate operation
    if gate_unitary!="cx":
        gate=eval(gate_unitary) #take the gate
        if target_qubits==[0]: #if we want to operate the first one
            I0 = np.identity(2**(total_qubits-1)) 
            O = np.kron(gate,I0)         
        elif target_qubits==[total_qubits-1]:    #if we want to operate the last one
            I0 = np.identity(2**(total_qubits-1))
            O = np.kron(I0, gate)
        else: #inbetween
            I0 = np.identity(2**(my_circuit[0]['target'][0]))
            I1 = np.identity(2**(total_qubits-my_circuit[0]['target'][0]-1))
            O = np.kron(np.kron(I0, gate),I1)
            
    #cnot-qubit gate operation
    else:
    
        c=target_qubits[0]
        t=target_qubits[1]
        # Define projection operator |0><0|
        P0x0 = np.array([[1, 0],[0, 0]])
        # Define projection operator |1><1|
        P1x1 = np.array([[0, 0],[0, 1]])
    

        if c==0:#if we want to operate the first one
            I0 = np.identity(2**(t-c-1))
            I1 = np.identity(2**(total_qubits-t-1))
            cn= np.kron(np.kron(P0x0, I0), I) + np.kron(np.kron(P1x1, I0), X)    
            O= np.kron(cn,I1)    
        
        elif  c==total_qubits-1:   #if we want to operate the last one 
            I0 = np.identity(2**(c-t-1))
            I1 = np.identity(2**(t))
            cn= np.kron(np.kron(I,I0), P0x0) + np.kron(np.kron(X,I0),P1x1)         
            O= np.kron(I1,cn)     
    
        else: #inbetween
            I0 = np.identity(2**(abs(c-t)-1))
            I1 = np.identity(2**(total_qubits-max(c,t)-1))
            I2 = np.identity(2**(min(c,t)))
            if c>t :
                cn= np.kron(np.kron(I,I0), P0x0) + np.kron(np.kron(X,I0),P1x1)         
            else:
                cn= np.kron(np.kron(P0x0, I0), I) + np.kron(np.kron(P1x1, I0), X)    
            O= np.kron(np.kron(I2,cn),I1)

    return O

def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    O1=np.identity(2**(total_qubits))
    for x in range(len(program)):
        O=get_operator(total_qubits, my_circuit[x]['gate'], my_circuit[x]['target'])
        O1=np.dot(O,O1) #multiply the operation
    q1=np.dot(O1,initial_state)
    
    # return final state
    return q1

def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    
    n = 2**total_qubits                  
    samplelist=[]
    for i in range(n):
        samplelist.append('{:0{}b}'.format(i, n))
  
    randomList = random.choices(samplelist, weights=state_vector, k=1) 
    

    return randomList[0]

def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)
    result=[]
    for x in range(num_shots):
        result.append(measure_all(state_vector))

    count=Counter(result)
    
    return  count

In [472]:
# Define program:

my_circuit = [
{ "gate": "h", "target": [0] }, 
{ "gate": "cx", "target": [0, 1] }
]


# Create "quantum computer" with 2 qubits (this is actually just a vector :) )
num_qubits=2
my_qpu = get_ground_state(num_qubits)


# Run circuit

final_state = run_program(my_qpu, my_circuit)


# Read results

counts = get_counts(final_state, 1000)

print(counts)

# Should print something like:
# {
#   "00": 502,
#   "11": 498
# }

# Voila!

Counter({'0011': 514, '0000': 486})
